## ALS

## 0. Setup

In [48]:
%load_ext autotime

time: 226 µs (started: 2024-03-25 21:29:29 -03:00)


In [1]:
#---- Bibliotecas:

# Análise e manipulação de dados:

import pandas as pd
import numpy as np

# Visualização de dados:

import plotly.io as pio
import plotly.express as px

# Manuseio nos dados

from os import chdir

# ALS

from implicit import als

/home/rafa/Documentos/github/book-recommendation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#---- Mudando o diretório raiz para a pasta principal do projeto

chdir('../../')

#---- Template dos gráficos:

pio.templates.default = "plotly_white"

#---- Carregando as funções

from functions import *

## 1. Extração dos dados

In [3]:
#---- Dados de treino

train_data = load_sparse_csr('03-data/02-processed/04-ALS-train-data.npz')

train_data

<22569x158236 sparse matrix of type '<class 'numpy.int64'>'
	with 346011 stored elements in Compressed Sparse Row format>

In [4]:
#---- Dados de teste

df_test = pd.read_parquet('03-data/02-processed/02-test_data.parquet')

df_test.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN
0,276747,1885408226,7,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,25.0,iowa city,iowa,usa,5
1,276762,N3453124715,4,None,None,None,None,25.0,duisburg,nordrhein-westfalen,germany,4
2,276772,3596151465,10,Henry der Held.,Roddy Doyle,2001,"Fischer (Tb.), Frankfurt",NaN,bonn,nordrhein-westfalen,germany,3
3,276786,8478442588,6,El Elogio de La Sombra,Tanazaki,1998,Siruela,34.0,madrid,madrid,spain,4
4,276788,055310666X,10,False Memory,Dean R. Koontz,1999,Bantam Books,NaN,mentone,california,usa,3


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22568 entries, 0 to 22567
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   User-ID              22568 non-null  int64  
 1   ISBN                 22568 non-null  object 
 2   Book-Rating          22568 non-null  int64  
 3   Book-Title           18886 non-null  object 
 4   Book-Author          18886 non-null  object 
 5   Year-Of-Publication  18886 non-null  object 
 6   Publisher            18886 non-null  object 
 7   Age                  15357 non-null  float64
 8   city                 22568 non-null  object 
 9   state                22568 non-null  object 
 10  country              22568 non-null  object 
 11  RN                   22568 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 2.1+ MB


In [6]:
#---- Dados de treino

df_train = pd.read_parquet('03-data/02-processed/05-ALS-df-train-data.parquet')

df_train.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN,USER_ID_CAT,IBSN_ID_CAT,USER_ID_INT,ISBN_ID_INT
0,276747,0060517794,9,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,25.0,iowa city,iowa,usa,1,22404,3844,1,1
1,276747,0671537458,9,Waiting to Exhale,Terry McMillan,1995,Pocket,25.0,iowa city,iowa,usa,2,22404,78067,1,2
2,276747,0679776818,8,Birdsong: A Novel of Love and War,Sebastian Faulks,1997,Vintage Books USA,25.0,iowa city,iowa,usa,3,22404,83361,1,3
3,276747,0943066433,7,How to Deal With Difficult People,Rick Brinkman,1995,Careertrack Inc.,25.0,iowa city,iowa,usa,4,22404,118092,1,4
4,276762,0380711524,5,See Jane Run,Joy Fielding,1992,Avon,25.0,duisburg,nordrhein-westfalen,germany,1,22405,37575,2,5


In [7]:
# #---- De-para's:

# de_para_isbn = pd.read_parquet('03-data/02-processed/05-ALS-DE-PARA-ISBN.parquet')
# de_para_user = pd.read_parquet('03-data/02-processed/06-ALS-DE-PARA-USER.parquet')

In [8]:
# #---- Full data:

# df_full = pd.read_parquet('03-data/02-processed/03-full_data.parquet')

# df_full.head()

## 2. Modelagem

In [9]:
model = als.AlternatingLeastSquares(factors = 30, random_state = 42)

model

/home/rafa/Documentos/github/book-recommendation/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [10]:
model.fit(train_data)

100%|███████████████████████████████████████████| 15/15 [02:05<00:00,  8.36s/it]


## 3. Recomendações

In [46]:
user_id_rec = 171011

de_para_user_id_rec = df_train.query(f'`User-ID` == {user_id_rec}')['USER_ID_INT'].iloc[0]

de_para_user_id_rec

1

In [12]:
#---- TOP 10 livros mais bem avaliados pelo usuário

df_train\
    .query(f'`User-ID` == {user_id_rec}')\
    .query(f'`Book-Rating` != 0')\
    .sort_values(by = 'Book-Rating', ascending = False)[['ISBN', 'Book-Title', 'Book-Rating']]\
    .drop_duplicates()\
    .head(10)

,ISBN,Book-Title,Book-Rating
221525,0439064872,Harry Potter and the Chamber of Secrets (Book 2),10
221526,0439136369,Harry Potter and the Prisoner of Azkaban (Book 3),10
221527,0439139597,Harry Potter and the Goblet of Fire (Book 4),10
221528,043935806X,Harry Potter and the Order of the Phoenix (Boo...,10
221529,043936213X,Harry Potter and the Sorcerer's Stone (Book 1),10
221524,0140386335,The Neverending Story,9


In [13]:
#---- Lista de ISBN que o usuário já leu

readed_isbn_by_user = df_train\
    .query(f'`User-ID` == {user_id_rec}')['Book-Title']\
    .unique()\
    .tolist()

readed_isbn_by_user

['The Neverending Story',
 'Harry Potter and the Chamber of Secrets (Book 2)',
 'Harry Potter and the Prisoner of Azkaban (Book 3)',
 'Harry Potter and the Goblet of Fire (Book 4)',
 'Harry Potter and the Order of the Phoenix (Book 5)',
 "Harry Potter and the Sorcerer's Stone (Book 1)"]

In [14]:
not_recommended_list_to_user = df_train\
    .query(f'`Book-Title`.isin({readed_isbn_by_user})')['ISBN_ID_INT']\
    .unique()\
    .tolist()

In [15]:
#---- Top10 recomendações para o usuário

recommendations = model.recommend(de_para_user_id_rec,
                                  train_data[de_para_user_id_rec], 
                                  filter_already_liked_items = True,
                                  N = 10,
                                  filter_items = not_recommended_list_to_user)

print(f'Recomendações: {recommendations}')

list_recommendations = recommendations[0].tolist()

#---- De-para das recomendações para ISBN

isbn_recommendations = df_train\
    .query(f'ISBN_ID_INT.isin({list_recommendations})')['ISBN']\
    .unique()\
    .tolist()

df_train\
    .pipe(filter_list_ordered, 
          col = 'ISBN', 
          list_filter = isbn_recommendations)\
    [['ISBN', 'Book-Title']]\
    .drop_duplicates()['Book-Title'].unique()

Recomendações: (array([ 1623,   810,  1478,  2081,  2077, 19063,  2529,   200,  1316,
         305], dtype=int32), array([0.6973636 , 0.3413098 , 0.31499463, 0.2582363 , 0.24672447,
       0.24356365, 0.24171497, 0.23169836, 0.2124185 , 0.20761147],
      dtype=float32))


array(['To Kill a Mockingbird', 'Fahrenheit 451',
       'The Hobbit : The Enchanting Prelude to The Lord of the Rings',
       'Angels &amp; Demons',
       'The Fellowship of the Ring (The Lord of the Rings, Part 1)',
       "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
       'The Catcher in the Rye',
       'The Two Towers (The Lord of the Rings, Part 2)',
       'The Bad Beginning (A Series of Unfortunate Events, Book 1)',
       'The Return of the King (The Lord of the Rings, Part 3)'],
      dtype=object)

## 4. Avaliação das métricas

In [ ]:
# Criar uma função que olhe a coluna User-ID e faça as TOP N recomendações. Resultado: lista
# Validar se dentro dessa lista de recomendações, está o livro-teste
# Criaremos uma flag: se o livro está dentro das recomendações, 1, senão 0
# Proporção dos valores 1 (Precision @ K)

In [67]:
#---- Criando uma função para recomendar K itens a partir do nome do usuário

def choose_books_als(user_id_to_rec: str, df_de_para: pd.DataFrame, rec_matrix, n: int = 10):

    # User_id para fazer a recomendação:

    user_id_de_para = df_de_para\
        .query(f'`User-ID` == {user_id_to_rec}')['USER_ID_INT'].iloc[0]

    recommendations = model.recommend(user_id_de_para,
                                      rec_matrix[user_id_de_para], 
                                      filter_already_liked_items = True,
                                      N = n)

    list_recommendations = recommendations[0].tolist()

    #---- De-para das recomendações para ISBN

    isbn_recommendations = df_de_para\
        .query(f'ISBN_ID_INT.isin({list_recommendations})')['ISBN']\
        .unique()\
        .tolist()
    
    list_recommendations = df_train\
        .pipe(filter_list_ordered, 
              col = 'ISBN', 
              list_filter = isbn_recommendations)\
        [['ISBN', 'Book-Title']]\
        .drop_duplicates()['ISBN'].unique()

    return list_recommendations

time: 3.91 ms (started: 2024-03-25 21:42:16 -03:00)


In [68]:
choose_books_als(user_id_to_rec = 171011, df_de_para = df_train, rec_matrix = train_data)

array(['0345339681', '0439139600', '0345339703', '059035342X',
       '0439064864', '0439136350', '0590353403', '0316769487',
       '0345339711', '0345339738'], dtype=object)

time: 613 ms (started: 2024-03-25 21:42:16 -03:00)


In [71]:
df_test_copy = df_test\
    .sample(10)

time: 1.68 ms (started: 2024-03-25 21:47:16 -03:00)


In [72]:
df_test_copy['ALEATORIO'] = df_test_copy\
    .apply(lambda x: choose_books_als(user_id_to_rec = x['User-ID'], 
                                      df_de_para = df_train, 
                                      rec_matrix = train_data), axis = 1)

time: 4.42 s (started: 2024-03-25 21:47:17 -03:00)


In [73]:
df_test_copy\
    .head(5)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN,ALEATORIO
14113,171104,1551923548,8,Rhymes With Useless,Terence Young,2001,Laurel Glen,NaN,port moody,british columbia,canada,5,"[0316776963, 0385492081, 0375725784, 067151005..."
11918,143609,0812510607,6,Good Behavior,Donald E. Westlake,1988,Tor Books (Mm),NaN,seattle,washington,usa,11,"[0440211727, 0345337662, 0060928336, 034538446..."
18121,222427,3257056125,6,Der Geliebte der Mutter.,Urs Widmer,2002,Diogenes,25.0,carouge,geneva,switzerland,11,"[0345342968, 0679723161, 0345337662, 051513287..."
10485,126054,1576732207,8,Heart's Delight - Valentine Anthology (Palisad...,Karen Ball,1997,Palisades,NaN,syracuse,new york,usa,3,"[0385504209, 0671021001, 0312195516, 031666634..."
17489,213873,091581109X,10,Creating Money: Keys to Abundance (Life Master...,Sanaya Roman,1988,H.J. Kramer,57.0,vancouver,british columbia,canada,5,"[0440241073, 0312195516, 0440224764, 031666634..."


time: 18.5 ms (started: 2024-03-25 21:47:21 -03:00)


# Para a próxima rodada de estudo:

- Otimizar o código
- Continuar a avaliação da métrica

# TO-DO's

- Ler um pouco mais sobre a teoria do ALS
- Criar um código que exclua o primeiro item da recomendação do usuário

# Validações

In [ ]:
df_train['FLAG_HP'] = np.where(df_train['Book-Title'].str.contains('Potter'), 1, 0)

In [ ]:
test = df_train[['User-ID', 'FLAG_HP']]\
    .value_counts()\
    .reset_index()\
    .sort_values(by = ['User-ID', 'FLAG_HP'])

test['percent'] = test['count'] / test.groupby('User-ID')['count'].transform('sum') * 100

test\
    .sort_values(by = 'percent', ascending = False)\
    .query('FLAG_HP == 1 & count >= 3')\
    .head(20)

In [ ]:
df_train\
    .query('`User-ID` == 252829')\
    .sort_values(by = 'Book-Title')